In [ ]:
#Code for improved fMRI-bias learning
#OV Jan 2021
####IMPORTS
import tensorflow as tf
import socket
import neurite as ne
import neurite_sandbox as nes
import numpy as np
import keras
import freesurfer as fs
from keras.optimizers import Adam
from keras import backend as K
from keras import layers as KL
#Load my own functions
import my_utils as utils
#from freesurfer import deeplearn as fsd
#OV: This imports all the local settings, e.g. subjects etc. from the header.py and netparms.py file, make your changes in there as needed
from netparms import *
from header_better import *
import losspad
#OV add matplotlib for plotting and cross checks
#import matplotlib.pyplot as plt
import random as rd
import string as string
####END IMPORTS
tf.__version__

READ IN DATA


In [ ]:
surfs_sphere   = np.load('data_train/surfs_sphere.npy',allow_pickle = "true")
surfs_pials    = np.load('data_train/surfs_pials.npy',allow_pickle = "true")
surfs_inflated = np.load('data_train/surfs_inflated.npy',allow_pickle = "true")
labels_cortex  = np.load('data_train/labels_cortex.npy',allow_pickle = "true")
maps_func      = np.load('data_train/maps_func.npy',allow_pickle = "true")
maps_angles    = np.load('data_train/maps_angles.npy',allow_pickle = "true")
maps_thickness = np.load('data_train/maps_thickness.npy',allow_pickle = "true")
maps_angio     = np.load('data_train/maps_angio.npy',allow_pickle = "true")
NumFuncs       = np.load('data_train/NumFuncs.npy',allow_pickle = "true")
#%whos

Define in and ouput data and assign a cortex label if wanted:

In [ ]:
numInputs = 3
#Percentile for removal of outliers
percentiles = [3,97]
#Assign in- and output data
inputs  = []
outputs = [] 
input_shape  = (maps_thickness[0][0].shape+(numInputs,))

for sno, subject in enumerate(subjects): 
   for hno, hemi in enumerate(hemis):
      for rn, func_runs in enumerate(Func_runs):
         if numInputs > 1:
            #We don't have a full set of runs for all subjects, some didn't register well
            #also multiply by cortex label 
            tmp_in = np.zeros(input_shape)
            try:
               #remove outliers and then normalise fluctuations and multiply by 100 
               tmp = maps_func[sno][hno][rn].data
               tmp = utils.zero_outliers_image(tmp,percentiles)
               tmp = utils.normalize_image(tmp)*100
               tmp = np.multiply(tmp,labels_cortex[sno][hno])
               outputs.append(tmp)
               tmp_in[...,0] = np.multiply(maps_thickness[sno][hno].data,labels_cortex[sno][hno])
               tmp_in[...,1] = np.multiply(maps_angio[sno][hno].data,labels_cortex[sno][hno])
               tmp_in[...,2] = np.multiply(maps_angles[sno][hno][rn].data,labels_cortex[sno][hno])
               inputs.append(tmp_in)
            except IndexError:
               continue
         elif numInputs == 1:
            try: 
               #remove outliers and then normalise fluctuations and multiply by 100 
               tmp = maps_func[sno][hno][rn].data
               tmp = utils.zero_outliers_image(tmp,percentiles)
               tmp = utils.normalize_image(tmp)*100
               tmp = np.multiply(tmp,labels_cortex[sno][hno])
               outputs.append(tmp)
               #inputs.append(np.multiply(maps_angio[sno][hno].data,labels_cortex[sno][hno]))
               inputs.append(np.multiply(maps_angles[sno][hno][rn].data,labels_cortex[sno][hno]))
               #inputs.append(np.multiply(maps_thickness[sno][hno].data,labels_cortex[sno][hno]))
            except IndexError:
               continue  
print(NumFuncs)


In [ ]:
im = utils.zero_outliers_image(maps_angles[0][0][0].data,percentiles)
plt.figure()
plt.imshow(tmp, cmap = 'jet')
plt.colorbar()
plt.figure()
plt.imshow(maps_func[0][0][0], cmap = 'jet')
plt.colorbar()

Circuarly leave-on-out strategy for training and testing

In [ ]:
#Define training settings
epochs        = 200
learning_rate = 1e-3
use_mask      = True

#Initialize
loss           = []
val_loss       = []
Prediction_err = []
NullOutput_err = []
NullInput_err  = []
plt.figure()

NumFuncs    = NumFuncs.astype(int)
CumSumFuncs = np.cumsum(NumFuncs)
for subj, subject in enumerate(subjects):
   target_shape = (maps_func[0][0][0].shape+(1,)) 
   input_shape  = (maps_thickness[0][0].shape+(numInputs,))
   print('subject: %s' %(subject))
  
   #OV: neuron (ne) expects another channel, but here we just have one number (the thickness or fmri derived parameter), 
   #maybe replace that with the time series itself? for now we just add np.newaxis as an empty channel
   #Assign training and testing data depending on subject
   if numInputs > 1: 
      #print(CumSumFuncs[subj-1],CumSumFuncs[subj])
      if (subj > 0 and subj<(len(subjects)-1)):
        xtrain    = np.array(np.concatenate((inputs[0 : CumSumFuncs[subj-1]],\
                                        inputs[CumSumFuncs[subj]:-1])))
        ytrain    = np.array(np.concatenate((outputs[0 : CumSumFuncs[subj-1]],\
                                        outputs[CumSumFuncs[subj]:-1])))[...,np.newaxis]
        xtest     = np.array(inputs[CumSumFuncs[subj-1] : CumSumFuncs[subj]])
        ytest     = np.array(outputs[CumSumFuncs[subj-1] : CumSumFuncs[subj]])[...,np.newaxis]
      elif subj == 0:
        xtrain    = np.array(inputs[NumFuncs[1]:-1])
        ytrain    = np.array(outputs[NumFuncs[1]:-1])[...,np.newaxis]
        xtest     = np.array(inputs[0:NumFuncs[0]])
        ytest     = np.array(outputs[0:NumFuncs[0]])[...,np.newaxis]
      elif subj == (len(subjects)-1):
        xtrain    = np.array(inputs[0:CumSumFuncs[subj-2]])
        ytrain    = np.array(outputs[0:CumSumFuncs[subj-2]])[...,np.newaxis]
        xtest     = np.array(inputs[CumSumFuncs[subj-1]:CumSumFuncs[subj]])
        ytest     = np.array(outputs[CumSumFuncs[subj-1]:CumSumFuncs[subj]])[...,np.newaxis]
                                     
   elif numInputs == 1:
      #print(CumSumFuncs[subj-1],CumSumFuncs[subj])
      if (subj > 0 and subj<(len(subjects)-1)):
        xtrain    = np.array(np.concatenate((inputs[0 : CumSumFuncs[subj-1]],\
                                        inputs[CumSumFuncs[subj]:-1])))[...,np.newaxis]
        ytrain    = np.array(np.concatenate((outputs[0 : CumSumFuncs[subj-1]],\
                                        outputs[CumSumFuncs[subj]:-1])))[...,np.newaxis]
        xtest     = np.array(inputs[CumSumFuncs[subj-1] : CumSumFuncs[subj]])[...,np.newaxis]
        ytest     = np.array(outputs[CumSumFuncs[subj-1] : CumSumFuncs[subj]])[...,np.newaxis]
      elif subj == 0:
        xtrain    = np.array(inputs[NumFuncs[1]:-1])[...,np.newaxis]
        ytrain    = np.array(outputs[NumFuncs[1]:-1])[...,np.newaxis]
        xtest     = np.array(inputs[0:NumFuncs[0]])[...,np.newaxis]
        ytest     = np.array(outputs[0:NumFuncs[0]])[...,np.newaxis]
      elif subj == (len(subjects)-1):
        xtrain    = np.array(inputs[0:CumSumFuncs[subj-2]])[...,np.newaxis]
        ytrain    = np.array(outputs[0:CumSumFuncs[subj-2]])[...,np.newaxis]
        xtest     = np.array(inputs[CumSumFuncs[subj-1]:CumSumFuncs[subj]])[...,np.newaxis] 
        ytest     = np.array(outputs[CumSumFuncs[subj-1]:CumSumFuncs[subj]])[...,np.newaxis]     

   print('input dimensions training: %s and testing %s ' %(xtrain.shape, xtest.shape))
   print('output dimensions: training %s and testing: %s ' %(ytrain.shape, ytest.shape))
    #sanity check
   #xtrain[xtrain>0] = 1
   #xtest[xtest>0]=1
   #Set up model
   K.clear_session()
   model = ne.models.unet(nb_features, input_shape, nb_depth, (3,3), 1, nb_conv_per_level = nb_conv_per_level, \
                          batch_norm = -1, final_pred_activation = 'linear')
   #make directory for callbacks 
   model_dir = 'model_dir'
   RandomName='fluc_from_triple'.join(rd.sample(string.ascii_lowercase,5))
   os.mkdir(os.path.join('/cluster/visuo/users/olivia/Projects/DeepBias/Better_code',model_dir,RandomName))
   save_file_name = os.path.join(model_dir, RandomName, '{epoch:02d}.h5') 
   save_callbacks = tf.keras.callbacks.ModelCheckpoint(save_file_name)
   #subclass of lr_callback written by Bruce. If loss explodes it detects that and falls back to a checkpoint
   lr_callbacks   = nes.tf.callbacks.ReduceLRWithModelCheckpointAndRecovery(save_file_name, \
                                                                            monitor='loss', \
                                                                            verbose=1, \
                                                                            cooldown=10, \
                                                                            factor=.5, \
                                                                            patience=100, \
                                                                            thres=.2, \
                                                                            save_weights_only=True, \
                                                                            min_lr=1e-7)

   sphereloss = losspad.spherical_loss(target_shape[0:2],pad=pad) #_no_pad[0:2],pad=pad)
   if use_mask == False:
       print('NOT using masked loss')
       #OV changed to tensorflow.keras bc it gave attribute error otherwise
       model.compile(optimizer=tf.keras.optimizers.Adam(lr=learning_rate), loss='MSE', metrics = ['accuracy'])
   else:
       print('using masked loss')
       model.compile(optimizer=Adam(lr=learning_rate), loss=utils.masked_MSE_loss , metrics = ['accuracy'])

   #Fit
   #dg = generator(xtrain[...,np.newaxis], ytrain, 8,augment_noise=.3)
   #Threshold thickness for training, but not for testing data
   batch_size = 8
   steps_per_epoch = xtrain.shape[0]/batch_size 
   dg    = utils.generator(xtrain, ytrain, batch_size = batch_size, augment_noise=.3)
   fhist = model.fit(dg, 
                     steps_per_epoch=steps_per_epoch, \
                     epochs=epochs, \
                     validation_data= (xtest, ytest), \
                     verbose=1, \
                     callbacks=[lr_callbacks])

   #Saving information for each CNN (i.e. each subject/cross valdiation cycle)
   loss.append(fhist.history['loss'])
   val_loss.append(fhist.history['val_loss'])
   ep = np.arange(epochs)
   plt.plot(ep,fhist.history['loss'], label=('train loss ' + subject))
   plt.plot(ep,fhist.history['val_loss'],label = ('val loss ' + subject))


   #PREDICT
   #Test on unseen data from unseen subject
   yp                 = model.predict(xtest)
   #Take out non cortex labeled areas
   if numInputs > 1:
      yp[xtest[:,:,:,0]==0]       = 0
      ytest[xtest[:,:,:,0]==0]    = 0
   else :
      yp[xtest==0]       = 0
      ytest[xtest==0]    = 0

   #Mean differene between predicition and truth
   print('Prediciting error is ' + str(np.absolute(ytest-yp).mean()))
   Prediction_err.append(np.absolute(ytest-yp).mean())

   #Error for scrambled prediction, "null output error"
   NullOutput   = ytest.copy()
   np.random.shuffle(NullOutput)
   #np.take(NullOutput,np.random.permutation(NullOutput.shape[2]),axis=2,out=NullOutput)
   NullOutput_err.append(np.absolute(ytest - NullOutput).mean())
   print('NullOutput error is ' + str(np.absolute(ytest - NullOutput).mean()))

   #Error for scrambled predictor (input), "null input error"
   NullInput    = xtest.copy().squeeze()
   np.random.shuffle(NullInput)
   #np.take(NullInput,np.random.permutation(NullInput.shape[2]),axis=2,out=NullInput)
   yp_NullInput = model.predict(NullInput)
   NullInput_err.append(np.absolute(ytest - yp_NullInput).mean())
   print('NullInput error is ' + str(np.absolute(ytest -yp_NullInput).mean()))

   #Visualize activations
   #extracts output of the top 8 layers
   #layer_outputs = [layer.output for layer in model.layers]
   #Creates a model that returns these outputs, given the model input:
   #activation_model = tf.keras.models.Model(inputs=model.input, outputs = layer_outputs)
   #this will return  list of 5 numpy arrays, one array per lyer activation
   #print(xtest.shape)
   #activations = activation_model.predict(xtest[0])
   #first_layer_activation = activations[0]
   #plt.matshow(first_layer_activation[0,:,:,11], cmap = 'viridis')
   #plt.show()


In [ ]:
print(xtest[:,:,:,0].shape)
test = ytest.copy().squeeze()
print(test[1,:10,1])
#np.random.shuffle(test)
#np.take(test,np.random.shuffle,axis=2,out=test)
print(test[1,:10,1])
print(ytest[1,:10,1])
print(yp[1,:10,1])
print(yp_NullInput[1,:10,1])

In [ ]:

#Histogram of predicition error for orientation (not in percent)
fig = plt.figure()
tmp = ytest-yp
tmp = tmp[~np.isnan(tmp)]
tmp = tmp[~np.isinf(tmp)]
tmp = tmp[tmp!=0]
tmp2 = ytest - NullOutput
tmp2 = tmp2[~np.isnan(tmp2)]
tmp2 = tmp2[~np.isinf(tmp2)] 
n,bins,patches = plt.hist(tmp, bins = 'auto', alpha = 0.4, label = 'Prediction')
plt.hist(tmp2, bins = bins, alpha = 0.4, label = 'Randomised input')
plt.ylim(0,40000)
plt.xlim(-30,30)
plt.xlabel('prediction error ')
plt.ylabel('counts')
#plt.xlim(-90,90)
plt.legend()
plt.title('Prediction error of orientation from fMRI')
plt.show()

In [ ]:
#Give out CNN performance results from cross-validation
print('CNN average prediction error: '+  str(np.array(Prediction_err).mean()) \
                                      + ' and std dev: ' + str(np.array(Prediction_err).std()))
print('CNN average NullInput error: ' +  str(np.array(NullInput_err).mean()) \
                                      + ' and std dev: ' + str(np.array(NullInput_err).std()))
print('CNN average NullOutput error: '+  str(np.array(NullOutput_err).mean()) \
                                      + ' and std dev: ' + str(np.array(NullOutput_err).std()))
#Show exemplar results
sphere      = surfs_sphere[-1][-1]
pial_disp   = surfs_inflated[-1][-1]
fv          = fs.Freeview()
truth       = sphere.sample_parameterization(ytest[-1,::].squeeze())
overlay     = fv.OverlayTag(truth, name='truth')#, threshold=(.0, 90)) 
overlays    = [overlay]
prediction  = sphere.sample_parameterization(yp[-1,::].squeeze())
overlay     = fv.OverlayTag(prediction, name='prediction')#, threshold=(0, 90)) 
overlays.append(overlay)
difference  = sphere.sample_parameterization(ytest[-1,::].squeeze()-yp[-1,::].squeeze())
overlay     = fv.OverlayTag(difference, name='difference')#, threshold=(-50,50)) 
overlays.append(overlay)


fv.surf(pial_disp, overlay=overlays)
fv.show(threads=20)
#fv.show.__code__.co_varnames

#Plot model overview
#keras.utils.plot_model(model,to_file='model.png',show_shapes=True,show_layer_names=True)

Plot binned dependencies on features

In [ ]:
import itertools
from scipy.stats import binned_statistic

tmpfunc      = list(itertools.chain(*maps_func[:][:][0][:]))
tmpfunc      = list(itertools.chain(*tmpfunc))
tmpangles    = list(itertools.chain(*maps_angles[:][:][0][:]))
tmpangles    = list(itertools.chain(*tmpangles))
tmpfunc      = np.array(tmpfunc)
tmpangles    = np.array(tmpangles)
tmpfunc      = tmpfunc.ravel()
tmpangles    = tmpangles.ravel()
tmpangio     = maps_angio.ravel()
tmpthick     = maps_thickness.ravel()

#Remove outliers from functional MRI
quant        = np.quantile(tmpfunc, [.1, .9])
mask         = np.ones(tmpfunc.shape)
mask[(tmpfunc>quant[1]) | (tmpfunc<quant[0])] = None 
tmpfunc      = tmpfunc[~np.isnan(mask)]
tmpangles    = tmpangles[~np.isnan(mask)]
tmpthick     = tmpthick[~np.isnan(mask)]
tmpangio     = tmpangio[~np.isnan(mask)]
plt.hist(tmpangio)

print(tmp.shape, tmp2.shape)
print(tmp,quant,mask,np.count_nonzero(mask) )
mean_angio = binned_statistic(tmpangio, tmpfunc, statistic='mean', bins=100, range=(0,2))
mean_thick = binned_statistic(tmpthick, tmpfunc, statistic='mean', bins=100, range=(0,5))
mean_angle = binned_statistic(tmpangles, tmpfunc, statistic='mean', bins=100, range=(0,90))
std_angio = binned_statistic(tmpangio, tmpfunc, statistic='std', bins=100, range=(0,2))
std_thick = binned_statistic(tmpthick, tmpfunc, statistic='std', bins=100, range=(0,5))
std_angle = binned_statistic(tmpangles, tmpfunc, statistic='std', bins=100, range=(0,90))

plt.figure()
plt.errorbar(mean_angio.bin_edges[:-1],mean_angio.statistic, label='fMRI binned by angio')
plt.figure()
plt.scatter(mean_thick.bin_edges[:-1],mean_thick.statistic, label='fMRI binned by thickness')
plt.figure()
plt.scatter(mean_angle.bin_edges[:-1],mean_angle.statistic, label='fMRI binned by angles')